In [1]:
import pandas as pd
import numpy as np
import pydataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings("ignore")

import env
import wrangle
import split_scale

> 1. Load the tips dataset.

In [2]:
tips = pydataset.data('tips')

>> a. Create a column named tip_percentage. This should be the tip amount divided by the total bill.

In [3]:
tips['tip_percentage'] = (tips.tip)/(tips.total_bill)
tips = tips.rename(columns={'size': 'party_size'})
tips

,total_bill,tip,sex,smoker,day,time,party_size,tip_percentage
1,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
2,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
3,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
4,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
5,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...,...
240,29.03,5.92,Male,No,Sat,Dinner,3,0.203927
241,27.18,2.00,Female,Yes,Sat,Dinner,2,0.073584
242,22.67,2.00,Male,Yes,Sat,Dinner,2,0.088222
243,17.82,1.75,Male,No,Sat,Dinner,2,0.098204


>> b. Create a column named price_per_person. This should be the total bill divided by the party size.

In [4]:
tips['price_per_person'] = (tips.total_bill)/(tips.party_size)
tips

,total_bill,tip,sex,smoker,day,time,party_size,tip_percentage,price_per_person
1,16.99,1.01,Female,No,Sun,Dinner,2,0.059447,8.495000
2,10.34,1.66,Male,No,Sun,Dinner,3,0.160542,3.446667
3,21.01,3.50,Male,No,Sun,Dinner,3,0.166587,7.003333
4,23.68,3.31,Male,No,Sun,Dinner,2,0.139780,11.840000
5,24.59,3.61,Female,No,Sun,Dinner,4,0.146808,6.147500
...,...,...,...,...,...,...,...,...,...
240,29.03,5.92,Male,No,Sat,Dinner,3,0.203927,9.676667
241,27.18,2.00,Female,Yes,Sat,Dinner,2,0.073584,13.590000
242,22.67,2.00,Male,Yes,Sat,Dinner,2,0.088222,11.335000
243,17.82,1.75,Male,No,Sat,Dinner,2,0.098204,8.910000


>> c. Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount? The tip percentage?

-The most important features for predicting the tip amount would be the size of the table, the time of day they eat, the day of the week, and the total bill

>> d. Use all the other numeric features to predict tip amount. Use select k best and recursive feature elimination to select the top 2 features. What are they?

In [5]:
# train, test = split_scale.split_data(tips)
# train.head()

In [6]:
# test.head()

In [7]:
X_train = tips[['total_bill', 'party_size', 'tip_percentage', 'price_per_person']]
y_train = tips[['tip']]

In [8]:
# scaler, train, test = split_scale.standard_scaler(X_train, X_test)

In [9]:
X_train.head()

,total_bill,party_size,tip_percentage,price_per_person
1,16.99,2,0.059447,8.495000
2,10.34,3,0.160542,3.446667
3,21.01,3,0.166587,7.003333
4,23.68,2,0.139780,11.840000
5,24.59,4,0.146808,6.147500


- Initialize the f_selector object, which defines the test for scoring the features and the number of features we want to keep, i.e. k)

In [10]:
f_selector = SelectKBest(f_regression, k=2)

In [11]:
f_selector.fit(X_train, y_train)

SelectKBest(k=2, score_func=<function f_regression at 0x1a21851440>)

In [12]:
# select the k best features
X2 = f_selector.transform(X_train)

X2 = SelectKBest(f_regression, k = 2).fit_transform(X_train, y_train)
print(X2.shape)

X2[0:2]

(244, 2)


array([[16.99,  2.  ],
       [10.34,  3.  ]])

In [13]:
f_support = f_selector.get_support()
f_support

array([ True,  True, False, False])

In [14]:
f_feature = X_train.loc[:,f_support].columns.tolist()
f_feature

['total_bill', 'party_size']

#### Recursive reature elimination

In [15]:
lm = LinearRegression()
rfe = RFE(lm, 2)
X_rfe = rfe.fit_transform(X_train, y_train)

In [16]:
mask = rfe.support_
rfe_features = X_train.loc[:,mask].columns.tolist()
rfe_features

['total_bill', 'tip_percentage']

In [17]:
var_ranks = rfe.ranking_
var_names = X_train.columns.tolist()

pd.DataFrame({'Feature': var_names, 'Rank': var_ranks})

,Feature,Rank
0,total_bill,1
1,party_size,2
2,tip_percentage,1
3,price_per_person,3


>> e. Use all the other numeric features to predict tip percentage. Use select k best and recursive feature elimination to select the top 2 features. What are they?

In [18]:
X_train2 = tips[['total_bill', 'party_size', 'tip_percentage', 'price_per_person']]
y_train = tips[['tip_percentage']]

In [19]:
X3 = SelectKBest(f_regression, k = 2).fit_transform(X_train2, y_train)
print(X3.shape)

(244, 2)


In [20]:
f_support = f_selector.get_support()
f_support

array([ True,  True, False, False])

In [21]:
f_feature = X_train.loc[:,f_support].columns.tolist()
f_feature

['total_bill', 'party_size']

#### Recursive

In [22]:
lm = LinearRegression()
rfe = RFE(lm, 2)
X_rfe = rfe.fit_transform(X_train2, y_train)

In [23]:
mask = rfe.support_
rfe_features = X_train.loc[:,mask].columns.tolist()
rfe_features

['party_size', 'tip_percentage']

In [24]:
var_ranks = rfe.ranking_
var_names = X_train.columns.tolist()

pd.DataFrame({'Feature': var_names, 'Rank': var_ranks})

,Feature,Rank
0,total_bill,3
1,party_size,1
2,tip_percentage,1
3,price_per_person,2


>> f. Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?

- I have no clue

> 2. Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

> 3. Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

> 4. Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).